<img src="data/lesson_1_stepik.png"></img>

In [1]:
import xml.etree.ElementTree as etree

## Список всех лемм

In [3]:
context = etree.iterparse("dict.opcorpora.xml", events=('end',))
odict = {}
pos = {}
i = 0

for event, tag_element in context:
    if tag_element.tag == 'lemma':
        word = tag_element.find('l').attrib['t']
        word_id = tag_element.attrib['id']
        pos[tag_element.attrib['id']] = (word, tag_element.find('l').find('g').attrib['v'])
        
        if word in odict.keys():
            odict[word].add(word_id)
        else:
            odict[word] = {word_id}
            
        for w in tag_element.findall('f'):
            word = w.attrib['t']
            if word in odict.keys():
                odict[word].add(word_id)
            else:
                odict[word] = {word_id}
        i += 1
        print(f"\r{i}", end="")

389424

## Частоты 

In [37]:
context = etree.iterparse("annot.opcorpora.no_ambig.xml", events=('end',))
w_freq = {}

for event, tag_element in context:
    if tag_element.tag == 'token':
        word = tag_element.attrib['text'].lower()
        w_id = tag_element.find('tfr').find('v').find('l').attrib['id']
        if w_id == '0':
            continue
        
        if word not in w_freq:
            w_freq[word] = {w_id: 1}
        elif w_id not in w_freq[word]:
            w_freq[word][w_id] = 1
        else:
            w_freq[word][w_id] += 1

In [10]:
pos['-1'] = ('unk', 'unk')

In [30]:
pos['0'] = ('«', 'PNCT')

In [11]:
pos_replace = {'NOUN': 'S',
               'COMP': 'A',
               'ADJF': 'A',
               'ADJS': 'A',
               'VERB': 'V',
               'INFN': 'V',
               'PRTF': 'V',
               'PRTS': 'V',
               'GRND': 'V',
               'PREP': 'PR',
               'CONJ': 'CONJ',
               'ADVB': 'ADV',
               'PRCL': 'ADV',
               'INTJ': 'ADV'}

## Исправление инфиннитива

In [12]:
pos_ids = list(pos)

In [13]:
for p_id in pos_ids:
    p = pos[p_id]
    if p[1] == 'INFN':
        pos[str(int(p_id)-1)] = p

## Решение для датасета

In [39]:
all_docs = []

with open('dataset.txt', encoding='utf-8') as f:
    for line in f:
        text = line.replace(',', '')
        text = text.replace('.', '')
        text = text.replace('!', '')
        text = text.replace('?', '')

        words = text.split()

        sentence = []
        for word in words:
            p = list(odict.get(word.lower()) or {'-1'})
            if len(p) > 1 and word in w_freq:
                p = max(zip(w_freq[word].values(), w_freq[word].keys()))[1]
            else:
                p = p[0]
            p = pos[p]
            cur_pos = pos_replace.get(p[1]) or p[1]
            sentence.append(f"{word}{{{p[0]}={cur_pos}}}")
        sentence = ' '.join(sentence)
        all_docs.append(sentence)

In [40]:
print('\n'.join(all_docs))

Выходит{выходить=V} абреки{абрек=S} необязательно{необязательно=ADV} гордые{гордый=A} мстители{мститель=S} и{и=CONJ} герои{герой=S} выходит{выходить=V} что{что=CONJ} они{они=NPRO} могут{мочь=V} сжечь{сжечь=V} сарай{сарай=S} или{или=CONJ} ни{ни=ADV} с{с=PR} того{тот=A} ни{ни=ADV} с{с=PR} сего{сие=NPRO} убить{убить=V} человека{человек=S}
Невозможно{невозможен=A} бегло{бегл=A} говорить{говорить=V} об{о=PR} этом{это=NPRO} но{но=CONJ} очевидно{очевидно=CONJ} богословие{богословие=S} Святого{святой=A} Духа{дух=S} имеет{иметь=V} отношение{отношение=S} к{к=PR} пониманию{понимание=S} Пушкина{пушкино=S}
Важный{важный=A} вывод{вывод=S} состоит{состоять=V} также{также=ADV} и{и=CONJ} в{в=PR} том{тот=A} что{что=CONJ} банковские{банковский=A} и{и=CONJ} почтовые{почтовый=A} системы{система=S} денежного{денежный=A} перевода{перевод=S} имеют{иметь=V} один{один=A} позитивный{позитивный=A} вектор{вектор=S} развития{развитие=S} и{и=CONJ} как{как=CONJ} ни{ни=ADV} странно{странно=ADV} именно{именно=ADV} конк